In [1]:
import pandas as pd
import numpy as np


In [11]:
df=pd.DataFrame(
{'key1':['a','a','b','b','a'],
 'key2':['one','two','one','two','one'],
 'data1':np.random.randint(1,10,5),
 'data2':np.random.randint(1,10,5)
}
)
df

,key1,key2,data1,data2
0,a,one,4,6
1,a,two,4,9
2,b,one,1,1
3,b,two,3,2
4,a,one,2,8


In [12]:
df.groupby('key1').sum()#内置求和函数

,data1,data2
key1,,
a,10,23
b,4,3


In [13]:
df.groupby('key1').mean()#求平均值

,data1,data2
key1,,
a,3.333333,7.666667
b,2.000000,1.500000


In [15]:
df.groupby('key1').count()

,key2,data1,data2
key1,,,
a,3,3,3
b,2,2,2


In [16]:
df.groupby('key1').size()

key1
a    3
b    2
dtype: int64

In [17]:
df.groupby('key1').max()
df.groupby('key1').min()

,key2,data1,data2
key1,,,
a,one,2,6
b,one,1,1


In [20]:
#还可以使用自定义的聚合函数
#可以使用hasattr函数来判断是否具有此属性
hasattr(df,'agg')

True

In [23]:
def peak_range(s):
    #s为一个可迭代对象
    print(type(s))
    return s.max()-s.min()
grouped=df.groupby('key1')
grouped.agg(peak_range)

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


,data1,data2
key1,,
a,2,3
b,2,1


In [29]:
#可以一次性应用多个聚合函数
grouped.agg(['std','mean','sum',peak_range])#自定义的聚合函数不能使用字符串
#默认使用函数名称作为列名称，可以使用元组修改列名称
grouped.agg(['std','mean','sum',('range',peak_range)])

<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>


data1                         data2                    
           std      mean sum range       std      mean sum range
key1                                                            
a     1.154701  3.333333  10     2  1.527525  7.666667  23     3
b     1.414214  2.000000   4     2  0.707107  1.500000   3     1

In [30]:
#对不同的列使用不同的聚合函数可以使用字典的方式
grouped.agg({
    'data1':[('平均值','mean'),('标准差','std')],
    'data2':[('最大值','max'),('最小值','min')]
})

data1           data2    
           平均值       标准差   最大值 最小值
key1                              
a     3.333333  1.154701     9   6
b     2.000000  1.414214     2   1

In [49]:
#可以通过使用merge函数将聚合运算的结果并入到原DataFrame中
#merge函数其实就是数据库里面的表链接
#add_prefix函数为每个label加上前缀
pd.merge(df,grouped.mean().add_prefix('mean_'),on='key1',right_index=True)

,key1,key2,data1,data2,mean_data1,mean_data2
0,a,one,4,6,3.333333,7.666667
1,a,two,4,9,3.333333,7.666667
4,a,one,2,8,3.333333,7.666667
2,b,one,1,1,2.000000,1.500000
3,b,two,3,2,2.000000,1.500000


In [52]:
#apply方法将一个函数作用在数据结构上,可以是df，grouper,与agg类似,可以更复杂的运算
df=pd.DataFrame(
{
    'key1':['a','a','b','b','a','a','a','b','b','a'],
    'key2':['one','two','one','two','one','one','two','one','two','one'],
    'data1':np.random.randint(1,10,10),
    'data2':np.random.randint(1,10,10)
})
df

,key1,key2,data1,data2
0,a,one,1,3
1,a,two,4,6
2,b,one,5,3
3,b,two,5,5
4,a,one,4,9
5,a,one,2,4
6,a,two,3,5
7,b,one,7,9
8,b,two,8,5
9,a,one,3,3


In [55]:
def top(g,n=2,column='data1'):
    return g.sort_values(by=column,ascending=False)[:n]
df.groupby('key1').apply(top)
df.groupby('key1').apply(top,n=3,column='data2')#后面的参数是传给top的所以可以实现比agg更复杂的功能

key1 key2  data1  data2
key1                          
a    4    a  one      4      9
     1    a  two      4      6
     6    a  two      3      5
b    7    b  one      7      9
     3    b  two      5      5
     8    b  two      8      5